In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [18]:
df = pd.read_json(
    "Sarcasm_Headlines_Dataset_v2.json",
    lines=True
)

df = df[["headline", "is_sarcastic"]]

texts = df["headline"].values
labels = df["is_sarcastic"].values


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)


In [20]:
VOCAB_SIZE = 20000
MAX_LEN = 40

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding="post")


In [21]:
model = Sequential()

model.add(
    Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=128,
        input_length=MAX_LEN
    )
)

model.add(
    Bidirectional(
        LSTM(64, return_sequences=False)
    )
)

model.add(Dropout(0.5))

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(1, activation="sigmoid"))

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [22]:
history = model.fit(
    X_train_pad,
    y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)


Epoch 1/15
287/287 [==============================] - 47s 142ms/step - loss: 0.4345 - accuracy: 0.7853 - val_loss: 0.3275 - val_accuracy: 0.8598
Epoch 2/15
287/287 [==============================] - 29s 101ms/step - loss: 0.1837 - accuracy: 0.9298 - val_loss: 0.3367 - val_accuracy: 0.8609
Epoch 3/15
287/287 [==============================] - 34s 117ms/step - loss: 0.0780 - accuracy: 0.9737 - val_loss: 0.5136 - val_accuracy: 0.8469
Epoch 4/15
287/287 [==============================] - 37s 129ms/step - loss: 0.0371 - accuracy: 0.9879 - val_loss: 0.6159 - val_accuracy: 0.8454
Epoch 5/15
287/287 [==============================] - 28s 99ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.8756 - val_accuracy: 0.8321
Epoch 6/15
287/287 [==============================] - 39s 135ms/step - loss: 0.0189 - accuracy: 0.9941 - val_loss: 0.8189 - val_accuracy: 0.8463
Epoch 7/15
287/287 [==============================] - 36s 127ms/step - loss: 0.0085 - accuracy: 0.9970 - val_loss: 0.9348 - val_acc

In [23]:
y_pred = (model.predict(X_test_pad) > 0.5).astype(int)

print(classification_report(y_test, y_pred))


179/179 [==============================] - 4s 16ms/step
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      2997
           1       0.83      0.80      0.82      2727

    accuracy                           0.83      5724
   macro avg       0.83      0.83      0.83      5724
weighted avg       0.83      0.83      0.83      5724



In [24]:
import pickle

# Save trained LSTM model
model.save("lstm_model")

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Model and tokenizer saved successfully")


INFO:tensorflow:Assets written to: lstm_model\assets


INFO:tensorflow:Assets written to: lstm_model\assets


✅ Model and tokenizer saved successfully
